# Sep 5-9, 17 2024: weighted graph -> weighted SBM

In [1]:
import csv
import os
import sys
import numpy as np
import pandas as pd
import scipy as sp 
import dill as pickle 
from os.path import join as pjoin
from itertools import product
from tqdm import tqdm
from copy import deepcopy
from pathlib import Path
import subprocess
from scipy import sparse, stats
from multiprocessing import Pool
import glob
import random

import arviz as az

from itertools import product, combinations
import multiprocessing as mp
from functools import partial

# networks
import graph_tool.all as gt

# plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.cm import rainbow

plt.rcParamsDefault['font.family'] = "sans-serif"
plt.rcParamsDefault['font.sans-serif'] = "Arial"
plt.rcParams['font.size'] = 14
plt.rcParams["errorbar.capsize"] = 0.5

import cmasher as cmr  # CITE ITS PAPER IN YOUR MANUSCRIPT
import colorcet as cc
from matplotlib.colors import LinearSegmentedColormap
from matplotlib import colors

# ignore user warnings
import warnings
warnings.filterwarnings("ignore") #, category=UserWarning)

In [2]:
class ARGS():
    pass

args = ARGS()

args.SEED = 100

def set_seed(args):
    gt.seed_rng(args.SEED)
    np.random.seed(args.SEED)

set_seed(args)

In [3]:
args.type = 'spatial'
args.roi_size = 225
args.maintain_symmetry = True
args.brain_div = 'whl'
args.num_rois = 162

PARC_DESC = (
    f'type-{args.type}'
    f'_size-{args.roi_size}'
    f'_symm-{args.maintain_symmetry}'
    f'_braindiv-{args.brain_div}'
    f'_nrois-{args.num_rois}'
)

In [4]:
args.GRAPH_DEF = f'constructed'
args.GRAPH_METHOD = f'pearson-corr'
args.THRESHOLDING = f'absolute'
args.EDGE_DEF = f'weighted'
args.EDGE_DENSITY = 10
args.LAYER_DEF = f'individual'
args.DATA_UNIT = f'ses'

BASE_path = f'{os.environ["HOME"]}/mouse_dataset'
PARCELS_path = f'{BASE_path}/parcels'
ROI_path = f'{BASE_path}/roi_results_v2/{PARC_DESC}'
TS_path = f'{ROI_path}/runwise_timeseries'
ROI_RESULTS_path = (
    f'{ROI_path}'
    f'/graph-{args.GRAPH_DEF}/method-{args.GRAPH_METHOD}'
    f'/threshold-{args.THRESHOLDING}/edge-{args.EDGE_DEF}/density-{args.EDGE_DENSITY}'
    f'/layer-{args.LAYER_DEF}/unit-{args.DATA_UNIT}'
)
GRAPH_path = f'{ROI_RESULTS_path}/graphs'
os.system(f'mkdir -p {GRAPH_path}')
SBM_path = f'{ROI_RESULTS_path}/model-fits'
os.system(f'mkdir -p {SBM_path}')
ESTIM_path = f'{ROI_RESULTS_path}/estimates'
os.system(f'mkdir -p {ESTIM_path}/individual')
os.system(f'mkdir -p {ESTIM_path}/group')

0

In [5]:
graphs_df = pd.read_csv(f'{ROI_RESULTS_path}/all_graphs.txt', names=['file'])
graphs_df['graph'] = graphs_df['file'].apply(lambda x: gt.load_graph(x))
graphs_df

,file,graph
0,/home/govindas/mouse_dataset/roi_results_v2/ty...,"<Graph object, undirected, with 162 vertices a..."
1,/home/govindas/mouse_dataset/roi_results_v2/ty...,"<Graph object, undirected, with 162 vertices a..."
2,/home/govindas/mouse_dataset/roi_results_v2/ty...,"<Graph object, undirected, with 162 vertices a..."
3,/home/govindas/mouse_dataset/roi_results_v2/ty...,"<Graph object, undirected, with 162 vertices a..."
4,/home/govindas/mouse_dataset/roi_results_v2/ty...,"<Graph object, undirected, with 162 vertices a..."
5,/home/govindas/mouse_dataset/roi_results_v2/ty...,"<Graph object, undirected, with 162 vertices a..."
6,/home/govindas/mouse_dataset/roi_results_v2/ty...,"<Graph object, undirected, with 162 vertices a..."
7,/home/govindas/mouse_dataset/roi_results_v2/ty...,"<Graph object, undirected, with 162 vertices a..."
8,/home/govindas/mouse_dataset/roi_results_v2/ty...,"<Graph object, undirected, with 162 vertices a..."
9,/home/govindas/mouse_dataset/roi_results_v2/ty...,"<Graph object, undirected, with 162 vertices a..."


In [6]:
g = graphs_df.iloc[0]['graph']
g

<Graph object, undirected, with 162 vertices and 1304 edges, 1 internal edge property, at 0x7fbd9445e5e0>

In [7]:
args.sbm = 'h'
args.dc = True

args.B = 122

In [8]:
def sbm_name(args):
    dc = f'dc' if args.dc else f'nd'
    dc = f'' if args.sbm in ['a'] else dc
    file = f'sbm-{dc}-{args.sbm}'
    return file

SBM = sbm_name(args)
SBM

'sbm-dc-h'

In [9]:
args.dc = True
args.sbm = 'd'
args.wait = 24000
args.force_niter = 40000
args.niter = 10
args.Bs = ' '.join([str(i) for i in (np.round(np.linspace(1, args.num_rois, 5)).astype(int))])

In [10]:
args.num_draws = int((1/2) * args.force_niter)

In [11]:
def sbm_name(args):
    dc = f'dc' if args.dc else f'nd'
    dc = f'' if args.sbm in ['a'] else dc
    file = f'sbm-{dc}-{args.sbm}'
    return file

SBM = sbm_name(args)
SBM

'sbm-dc-d'

In [12]:
files = sorted(glob.glob(f'{GRAPH_path}/*', recursive=True))
file = files[0]

In [13]:
g = gt.load_graph(file)
g

<Graph object, undirected, with 162 vertices and 1304 edges, 1 internal edge property, at 0x7fbd6db36af0>

In [14]:
args.Bs

'1 41 82 122 162'

In [15]:
args.transform = 'arctanh'

In [16]:
os.system((
    f'bash 03a-graph-individual-weighted_desc-equilibrate.sh '
    f'{file} ' 
    f'{args.sbm} ' 
    f'{args.dc} ' 
    f'{args.wait} ' 
    f'{args.force_niter} ' 
    f'"{args.Bs}" '
    f'{args.transform} '
    f'{args.SEED} '
))

41
chain equilibrated
posterior modes found
saved all files
82
chain equilibrated
posterior modes found
saved all files
162
chain equilibrated
posterior modes found
saved all files
122
chain equilibrated
posterior modes found
saved all files
1
chain equilibrated
posterior modes found
saved all files


0